# Fifth Agent Agent

Introducing a critical agent - the agent that brings it all together.

# Planning Agent

There are a number of frameworks out there that support building Agentic Workflows.

OpenAI has OpenAI Agents SDK, LangChain has LangGraph, and there's Autogen from Microsoft, Crew.ai and many others.  

Each of these are abstractions on top of APIs to LLMs; some are lightweight, others come with significant functionality.

# We are going to use OpenAI Agents SDK for this Agent

## And we're using Tools to give our Agent autonomy

In our case, we're going to create an Agent that uses Tools to make decisions about what to do next.

Let's see how it works

In [1]:
# imports

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import os
from agents.mcp import MCPServerStdio

In [2]:
!pip show mcp

Name: mcp
Version: 1.14.0
Summary: Model Context Protocol SDK
Home-page: https://modelcontextprotocol.io
Author: Anthropic, PBC.
Author-email: 
License: MIT
Location: /Users/ed/miniconda3/envs/tech2ai/lib/python3.11/site-packages
Requires: anyio, httpx, httpx-sse, jsonschema, pydantic, pydantic-settings, python-multipart, sse-starlette, starlette, uvicorn
Required-by: openai-agents


In [3]:
from agents import Agent, Runner, trace, function_tool

In [4]:
# Initialization

load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
MODEL = "gpt-4.1"


OpenAI API Key exists and begins sk-proj-


In [5]:
# Use the Scanner agent from before

from price_agents.scanner_agent import ScannerAgent
scanner = ScannerAgent()

# Our tools

The next 3 cells have 3 **fake** functions that we will allow our LLM to call

In [6]:
@function_tool
def scan_the_internet_for_bargains():
    """ This tool scans the internet for great deals and gets a curated list of promising deals """
    print(f"Scanning the internet")
    results = scanner.test_scan()
    return results.model_dump()

In [7]:
@function_tool
def estimate_true_value(description: str) -> str:
    """
    This tool estimates the true value of a product based on a text description of it

    Args:
        description: a description of the product to be estimated
    """
    print(f"Estimating true value of {description[:20]}...")
    result = {"description": description, "estimated_true_value": 300}
    return json.dumps(result)

In [8]:
@function_tool
def notify_user_of_deal(description: str, deal_price: float, estimated_true_value: float) -> str:
    """
    This tool notifies the user of a great deal, given a description of it, the price of the deal, and the estimated true value

    Args:
        description: a description of the product with the deal
        deal_price: how much the product is being offered for
        estimated_true_value: an estimate of how much this product is actually worth
        url: the web address of the product
    """
    print(f"Notifying user of {description} which costs {deal_price} and estimate is {estimated_true_value}")
    return "notification sent ok"

In [9]:
notify_user_of_deal.params_json_schema

{'properties': {'description': {'description': 'a description of the product with the deal',
   'title': 'Description',
   'type': 'string'},
  'deal_price': {'description': 'how much the product is being offered for',
   'title': 'Deal Price',
   'type': 'number'},
  'estimated_true_value': {'description': 'an estimate of how much this product is actually worth',
   'title': 'Estimated True Value',
   'type': 'number'}},
 'required': ['description', 'deal_price', 'estimated_true_value'],
 'title': 'notify_user_of_deal_args',
 'type': 'object',
 'additionalProperties': False}

## Telling the LLM about the tools it can use, with JSON

"Tool calling" is giving an LLM the power to run code on your computer!

Sounds a bit spooky?

The way it works is a little more mundane. We give the LLM a description of each Tool and the parameters, and we let it inform us if it wants any tool to be run.

It's not like OpenAI reaches in and runs a function. In the end, we have an if statement that calls our function if the model requests it.

## OpenAI Agents SDK has made this easy for us

The decorator `function_tools` around each of our functions automatically generates the description we need for the LLM

In [10]:
tools = [scan_the_internet_for_bargains, estimate_true_value, notify_user_of_deal]
tools

[FunctionTool(name='scan_the_internet_for_bargains', description='This tool scans the internet for great deals and gets a curated list of promising deals', params_json_schema={'properties': {}, 'title': 'scan_the_internet_for_bargains_args', 'type': 'object', 'additionalProperties': False, 'required': []}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x10db09440>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='estimate_true_value', description='This tool estimates the true value of a product based on a text description of it', params_json_schema={'properties': {'description': {'description': 'a description of the product to be estimated', 'title': 'Description', 'type': 'string'}}, 'required': ['description'], 'title': 'estimate_true_value_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x10dfe9b20>, strict_json_

## And.. MCP

The Model Context Protocol from Anthropic is causing a lot of excitement.

It gives us a really easy way to integrate new capabilities with our agent, as more tools.

Here we will give our agent powers to write to our local filesystem in a directory "sandbox"

MCP is covered in more detail in other sessions (linked in resources) but the basics:

1. You describe them using parameters
2. You create them with `with MCPServerStdio(params=params) as server`
3. You can call `server.list_tools()` or pass in `server` 

In [11]:
sandbox_path = os.path.abspath(os.path.join(os.getcwd(), "sandbox"))

# parameters describe an MCP server
files_params = {"command": "npx", "args": ["-y", "@modelcontextprotocol/server-filesystem", sandbox_path]}


async with MCPServerStdio(params=files_params, client_session_timeout_seconds=30) as server:
    file_tools = await server.session.list_tools()

In [12]:
file_tools.tools

[Tool(name='read_file', title=None, description='Read the complete contents of a file as text. DEPRECATED: Use read_text_file instead.', inputSchema={'type': 'object', 'properties': {'path': {'type': 'string'}, 'tail': {'type': 'number', 'description': 'If provided, returns only the last N lines of the file'}, 'head': {'type': 'number', 'description': 'If provided, returns only the first N lines of the file'}}, 'required': ['path'], 'additionalProperties': False, '$schema': 'http://json-schema.org/draft-07/schema#'}, outputSchema=None, annotations=None, meta=None),
 Tool(name='read_text_file', title=None, description="Read the complete contents of a file from the file system as text. Handles various text encodings and provides detailed error messages if the file cannot be read. Use this tool when you need to examine the contents of a single file. Use the 'head' parameter to read only the first N lines of a file, or the 'tail' parameter to read only the last N lines of a file. Operates 

In [13]:
system_message = "You are an Autonomous AI Agent that makes use of tools to carry out your mission. \
Your mission is to find great deals on bargain products, and notify the user when you find them with a push notification and a written file."

user_message = "Your mission is to discover great deals on products. First you should scan the internet for bargain deals. Then for each deal, you should estimate its true value - how much it's actually worth. "
user_message += "Finally, you should pick the single most compelling deal where the deal price is much lower than the estimated true value, and "
user_message += "use your tools to send the user a push notification about that deal, and also use your tools to write or update a file called sandbox/deals.md with a description in markdown. "
user_message += "You must only notify the user about one deal, and be sure to pick the most compelling deal, where the deal price is much lower than the estimated true value. Then just respond OK to indicate success."
messages = [{"role": "system", "content": system_message},{"role": "user", "content": user_message}]

### And here's where it comes together - just 2 lines of code

In [14]:
async with MCPServerStdio(params=files_params) as server:
    agent = Agent(name="Planner", instructions=system_message, model="gpt-5", tools=tools, mcp_servers=[server])
    result = await Runner.run(agent, user_message)

print(result.final_output)

Scanning the internet
Estimating true value of The Hisense R6 Serie...
Estimating true value of The Poly Studio P21 ...
Estimating true value of The Lenovo IdeaPad S...
Estimating true value of The Dell G15 gaming ...
Notifying user of Poly Studio P21 21.5-inch LED Personal Meeting Display with 1080p webcam, stereo speakers, privacy shutter, ambient light/vanity lighting, and 5W wireless charging — an all-in-one WFH videoconferencing monitor. which costs 30.0 and estimate is 300.0
OK


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/traces/ingest "HTTP/1.1 204 No Content"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/traces/ingest "HTTP/1.1 204 No Content"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/traces/ingest "HTTP/1.1 204 No Content"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/traces/ingest "HTTP/1.1 204 No Content"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/traces/ingest "HTTP/1.1 204 No Content"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/traces/ingest "HTTP/1.1 204 No Content"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/traces/ingest "HTTP/1.1 204 No Content"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/traces/ingest "HTTP/1.1 204 No Content"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/traces/ingest "HTTP/1.1 204 No Content"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/traces/ingest "HTTP/1.1 204 No Content"


## And now - putting all of this into a Planning Agent

In [15]:
from price_agents.autonomous_planning_agent import AutonomousPlanningAgent

In [16]:
import logging
root = logging.getLogger()
root.setLevel(logging.INFO)

In [17]:
import chromadb
DB = "products_vectorstore"
client = chromadb.PersistentClient(path=DB)
collection = client.get_or_create_collection('products')

INFO:chromadb.telemetry.product.posthog:Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.


In [18]:
agent = AutonomousPlanningAgent(collection)

INFO:root:[Autonomous Planning Agent] Autonomous Planning Agent is initializing
INFO:root:[Scanner Agent] Scanner Agent is initializing
INFO:root:[Scanner Agent] Scanner Agent is ready
INFO:root:[Frontier Agent] Initializing Frontier Agent
INFO:root:[Frontier Agent] Frontier Agent is set up with DeepSeek
INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: mps
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2
INFO:root:[Frontier Agent] Frontier Agent is ready
INFO:root:[Specialist Agent] Specialist Agent is initializing - connecting to modal
INFO:root:[Specialist Agent] Specialist Agent is ready
INFO:root:[Messaging Agent] Messaging Agent is initializing
INFO:root:[Messaging Agent] Messaging Agent has initialized Pushover and Claude
INFO:root:[Autonomous Planning Agent] Autonomous Planning Agent is ready


In [19]:
result = agent.plan()

INFO:root:[Autonomous Planning Agent] Autonomous Planning Agent is kicking off a run
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
INFO:root:[Autonomous Planning Agent] Autonomous Planning agent is calling scanner
INFO:root:[Scanner Agent] Scanner Agent is about to fetch deals from RSS feed
INFO:root:[Scanner Agent] Scanner Agent received 15 deals not already scraped
INFO:root:[Scanner Agent] Scanner Agent is calling OpenAI using Structured Output
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:[Scanner Agent] Scanner Agent received 5 selected deals with price>0 from OpenAI
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
INFO:root:[Autonomous Planning Agent] Autonomous Planning agent is estimating value
INFO:root:[Frontier Agent] Frontier Agent is using Llama 3.2 to preprocess the description
INFO:httpx:HTTP Request: POST http://localhost:11434/v1/chat/completi

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:root:[Frontier Agent] Frontier Agent is performing a RAG search of the Chroma datastore to find 5 similar products
INFO:root:[Frontier Agent] Frontier Agent has found similar products
INFO:root:[Frontier Agent] Frontier Agent is about to call DeepSeek with context including 5 similar products
INFO:httpx:HTTP Request: POST https://api.deepseek.com/chat/completions "HTTP/1.1 200 OK"
INFO:root:[Frontier Agent] Frontier Agent completed - predicting $1198.00
INFO:root:[Specialist Agent] Specialist Agent is calling remote fine-tuned model
INFO:root:[Specialist Agent] Specialist Agent completed - predicting $950.00
INFO:root:[Autonomous Planning Agent] Autonomous Planning agent is estimating value
INFO:root:[Frontier Agent] Frontier Agent is using Llama 3.2 to preprocess the description
INFO:httpx:HTTP Request: POST http://localhost:11434/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:[Frontier Agent] Frontier Agent is vectorizing using all-MiniLM-L6-v2


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:root:[Frontier Agent] Frontier Agent is performing a RAG search of the Chroma datastore to find 5 similar products
INFO:root:[Frontier Agent] Frontier Agent has found similar products
INFO:root:[Frontier Agent] Frontier Agent is about to call DeepSeek with context including 5 similar products
INFO:httpx:HTTP Request: POST https://api.deepseek.com/chat/completions "HTTP/1.1 200 OK"
INFO:root:[Frontier Agent] Frontier Agent completed - predicting $549.00
INFO:root:[Specialist Agent] Specialist Agent is calling remote fine-tuned model
INFO:root:[Specialist Agent] Specialist Agent completed - predicting $500.00
INFO:root:[Autonomous Planning Agent] Autonomous Planning agent is estimating value
INFO:root:[Frontier Agent] Frontier Agent is using Llama 3.2 to preprocess the description
INFO:httpx:HTTP Request: POST http://localhost:11434/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:[Frontier Agent] Frontier Agent is vectorizing using all-MiniLM-L6-v2


...

INFO:root:[Frontier Agent] Frontier Agent is performing a RAG search of the Chroma datastore to find 5 similar products
INFO:root:[Frontier Agent] Frontier Agent has found similar products
INFO:root:[Frontier Agent] Frontier Agent is about to call DeepSeek with context including 5 similar products
INFO:httpx:HTTP Request: POST https://api.deepseek.com/chat/completions "HTTP/1.1 200 OK"
INFO:root:[Frontier Agent] Frontier Agent completed - predicting $349.00
INFO:root:[Specialist Agent] Specialist Agent is calling remote fine-tuned model
INFO:root:[Specialist Agent] Specialist Agent completed - predicting $220.00
INFO:root:[Autonomous Planning Agent] Autonomous Planning agent is estimating value
INFO:root:[Frontier Agent] Frontier Agent is using Llama 3.2 to preprocess the description
INFO:httpx:HTTP Request: POST http://localhost:11434/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:[Frontier Agent] Frontier Agent is vectorizing using all-MiniLM-L6-v2


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:root:[Frontier Agent] Frontier Agent is performing a RAG search of the Chroma datastore to find 5 similar products
INFO:root:[Frontier Agent] Frontier Agent has found similar products
INFO:root:[Frontier Agent] Frontier Agent is about to call DeepSeek with context including 5 similar products
INFO:httpx:HTTP Request: POST https://api.deepseek.com/chat/completions "HTTP/1.1 200 OK"
INFO:root:[Frontier Agent] Frontier Agent completed - predicting $649.99
INFO:root:[Specialist Agent] Specialist Agent is calling remote fine-tuned model
INFO:root:[Specialist Agent] Specialist Agent completed - predicting $500.00
INFO:root:[Autonomous Planning Agent] Autonomous Planning agent is estimating value
INFO:root:[Frontier Agent] Frontier Agent is using Llama 3.2 to preprocess the description
INFO:httpx:HTTP Request: POST http://localhost:11434/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:[Frontier Agent] Frontier Agent is vectorizing using all-MiniLM-L6-v2


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:root:[Frontier Agent] Frontier Agent is performing a RAG search of the Chroma datastore to find 5 similar products
INFO:root:[Frontier Agent] Frontier Agent has found similar products
INFO:root:[Frontier Agent] Frontier Agent is about to call DeepSeek with context including 5 similar products
INFO:httpx:HTTP Request: POST https://api.deepseek.com/chat/completions "HTTP/1.1 200 OK"
INFO:root:[Frontier Agent] Frontier Agent completed - predicting $799.00
INFO:root:[Specialist Agent] Specialist Agent is calling remote fine-tuned model
INFO:root:[Specialist Agent] Specialist Agent completed - predicting $799.00
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
INFO:root:[Autonomous Planning Agent] Autonomous Planning agent is notifying user
INFO:root:[Messaging Agent] Messaging Agent is using Claude to craft the message
INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
INFO:root:[Messaging Agent] Messaging Agent is 

### Check out the trace

https://platform.openai.com/traces

# Finally - with a Gradio UI

In [20]:
# Reset memory back to 2 deals discovered in the past

from deal_agent_framework import DealAgentFramework
DealAgentFramework.reset_memory()

In [ ]:
!python price_is_right.py

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


* Running on local URL:  http://127.0.0.1:7863
* To create a public link, set `share=True` in `launch()`.
[2025-09-23 16:42:54 -0400] [Agents] [INFO] [Agent Framework] Initializing Agent Framework
[2025-09-23 16:42:54 -0400] [Agents] [INFO] [Agent Framework] Initializing Agent Framework
[2025-09-23 16:42:54 -0400] [Agents] [INFO] [Autonomous Planning Agent] Autonomous Planning Agent is initializing
[2025-09-23 16:42:54 -0400] [Agents] [INFO] [Autonomous Planning Agent] Autonomous Planning Agent is initializing
[2025-09-23 16:42:54 -0400] [Agents] [INFO] [Scanner Agent] Scanner Agent is initializing
[2025-09-23 16:42:54 -0400] [Agents] [INFO] [Scanner Agent] Scanner Agent is initializing
[2025-09-23 16:42:54 -0400] [Agents] [INFO] [Scanner Agent] Scanner Agent is ready
[2025-09-23 16:42:54 -0400] [Agents] [INFO] [Scanner Agent] Scanner Agent is ready
[2025-09-23 16:42:54 -0400] [Agents] [INFO] [Frontier Agent] Initializing Frontier Agent
[2025-09-23 16:42:54 -0400] [Agents] [INFO] [Fron